In [2]:
using CairoMakie, Dierckx, Optim, LinearAlgebra, QuantEcon

In [3]:
pars = (;α = 0.33, # Capital share
        β = 0.9, # Discount factor
        A = 10.0, # TFP
        γ = 1.0, # Risk aversion
        δ = 0.1, # Depreciation rate
        nk = 31, # Number of capital gridpoints
        θ = 3, # Grid expansion parameter
        lb = 0.00, # Lower bound of capital grid
        ub = 1000.0, # Upper bound of capital grid
        nz = 19, # Number of shock gridpoints
        ρ = 0.9, # Persistence of AR(1) process
        μ = 0.0, # Mean of AR(1) process
        σ = 0.007, # Var of AR(1) process
        toler = 1e-6, # Tolerance
        maxiter = 10000) # Maximum number of iterations

(α = 0.33, β = 0.9, A = 10.0, γ = 1.0, δ = 0.1, nk = 31, θ = 3, lb = 0.0, ub = 1000.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.007, toler = 1.0e-6, maxiter = 10000)

In [24]:
function utility(c)#, pars)
    #(; γ) = pars
    if pars.γ == 1.0
        return log(c)
    else
        return (c^(1-γ)) / (1-γ)
    end
end

function ar1(pars)
    (; μ, ρ, σ, nz) = pars
    mc = rouwenhorst(nz, μ, ρ, σ)
    Π, Zvals = mc.p, exp.(mc.state_values)
    return Π, Zvals
end

function exp_grid(pars)
    (; nk, θ, lb, ub) = pars
    grid = LinRange(1e-4, 1.0, nk)
    expgrid = lb .+ (ub .- lb) .* grid.^θ
    return expgrid
end

function interpolate(grid, vals, pars)
    spline = Spline1D(grid, vals, k = 3, bc = "extrapolate")
    return spline
end

function production(k, z ,pars)
    (; α, δ, A) = pars
    return A * z * (k ^ α) + (1.0 - δ) * k
end

function initial_guess(grid, pars)
    (; nk, nz) = pars
    v = ones(nk, nz)
    v_out = zeros(nk, nz)
    for j in 1:nz
        v_out[:,j] = utility.(grid)#, pars)
    end
    return v_out
end

function resource_grid(kgrid, zgrid, pars)
    (; nk, nz) = pars
    Ygrid = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            Ygrid[i,j] = production(kgrid[i], zgrid[j], pars)
        end
    end
    return Ygrid
end

#test_Π, test_Zvals = ar1(pars)
#test_grid = exp_grid(pars)
#test_v = ones(pars.nk)
#test_interp = interpolate(test_grid, test_v, pars)
#test_deriv = Dierckx.derivative(test_interp, 15.0)
#test_production = production(test_grid[19], test_Zvals[1], pars)
#test_init = initial_guess(test_grid, pars)

resource_grid (generic function with 1 method)

In [25]:
function egm(pars)
    (; nk, nz, toler, maxiter) = pars
    Π, Zvals = ar1(pars)
    Kvals = exp_grid(pars)
    v = initial_guess(exp_grid(pars), pars)
    v_new = zeros(nk, nz)
    c = zeros(nk, nz)
    derivatives = zeros(nk, nz)
    Yvals = resource_grid(Kvals, Zvals, pars)
    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end
    return #Yvals
    while ((error > toler) && (iter < maxiter))
end

test = egm(pars)
    

Iterating...


31×19 Matrix{Float64}:
   0.000236996    0.00036224    0.000553671  …     0.321429     0.491292
   0.108378       0.147883      0.208265         101.42       154.999
   0.416044       0.494392      0.614144         201.342      307.602
   1.12381        1.24079       1.41961          301.144      459.812
   2.43174        2.58724       2.82491          401.211      612.108
   4.53985        4.73376       5.03015      …   501.829      764.823
   7.64812        7.88036       8.23534          603.25       918.236
  11.9565        12.227        12.6405           705.705     1072.6
  17.665         17.9737       18.4456           809.421     1228.15
  24.9734        25.3203       25.8506           914.614     1385.1
   ⋮                                         ⋱               
 356.562        357.402       358.687           2511.47      3651.1
 407.265        408.143       409.485           2659.11      3850.0
 462.568        463.483       464.883           2811.3       4053.44
 522.669    